In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../..")

import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc

In [2]:
global_friction_file    = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1.geotiff"
global_friction_noOcean = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1_no_ocean_travel.tif"
ports_file = "/home/wb411133/data/Global/INFRA/PORTS/major_ports.shp"



In [ ]:
if not os.path.exists(global_friction_noOcean):# Burn the oceans in as a no travel zone
    inG = rasterio.open(global_friction_file)
    inG_data = inG.read()[0,:,:]
    inG_data = inG_data * 1000
    
    ocean_file = "/home/public/Data/GLOBAL/ADMIN/OCEAN/ne_10m_ocean.shp"
    cMeta = inG.meta.copy()

    inO = gpd.read_file(ocean_file)
    shapes = ((row.geometry,999999) for idx, row in inO.iterrows())
    burned = features.rasterize(shapes=shapes, out_shape=(cMeta['height'], cMeta['width']), 
                                transform=cMeta['transform'], dtype=cMeta['dtype'])
    inG_combo = inG_data + burned
    out_file = "/home/wb411133/temp/2015_friction_surface_v1_no_ocean_travel.tif"
    with rasterio.open(out_file, 'w', **cMeta) as out:
        out.write_band(1, inG_combo)
   

In [3]:
inG = rasterio.open(global_friction_noOcean)
inP = gpd.read_file(ports_file)

inG_data = inG.read()[0,:,:]
mcp = graph.MCP_Geometric(inG_data)

In [4]:
inP = gpd.read_file(ports_file)
inP.head()

,OBJECTID,Postal,Lat,Lng,Type,ISO3,REGION,geometry
0,12,"Mongla (Chalna),Bangladesh",22.488900,89.595800,Bulk,BGD,South Asia,POINT (89.59580 22.48890)
1,19,"Suape (Recife),Brazil",-8.055556,-34.891111,General,BRA,Latin America & Caribbean,POINT (-34.88870 -8.05143)
2,20,"Pecém (near Fortaleza),Brazil",-3.533611,-38.785833,General,BRA,Latin America & Caribbean,POINT (-38.81149 -3.54865)
3,23,"Ponta da Madeira,Brazil",-2.565000,-44.370000,Bulk,BRA,Latin America & Caribbean,POINT (-44.32862 -2.53360)
4,24,"Ponta do Ubu,Brazil",-20.788500,-40.573700,Bulk,BRA,Latin America & Caribbean,POINT (-40.62399 -20.82856)


# Optional processing of ports

In [ ]:
# Attach ISO3 codes and regions to ports
global_boundaries =  "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inB = gpd.read_file(global_boundaries)

In [ ]:
inP = gpd.read_file(ports_file)
if inP.crs != inB.crs:
    inP = inP.to_crs(inB.crs)
    
inP['ISO3'] = ''
inP['REGION'] = ''
for idx, row in inP.iterrows():
    sel_country = inB.loc[inB.intersects(row['geometry'])]
    if sel_country.shape[0] > 0:
        inP.loc[idx,'ISO3'] = sel_country['ISO3'].iloc[0]
        inP.loc[idx,'REGION'] = sel_country['Region'].iloc[0]
    else:
        print(f"{idx}: {row['Postal']}")
              
inP = inP.to_crs({'init':'epsg:4326'})
inP.to_file(ports_file)

In [ ]:
# Determine which ports are accidentally in the ocean
ocean_layer = "/home/public/Data/GLOBAL/ADMIN/OCEAN/ne_10m_ocean.shp"
inO = gpd.read_file(ocean_layer)
inO.head()

In [ ]:
inP = gpd.read_file(ports_file)

In [ ]:
inP_idx = inP['geometry'].apply(lambda x: x.intersects(inO['geometry'].iloc[0]))

In [ ]:
inP.loc[inP_idx].to_file(ports_file.replace(".shp", "_BAD.shp"))

In [ ]:
bad_p = gpd.read_file(ports_file.replace(".shp", "_BAD.shp"))
all_files = os.listdir(out_folder)
for idx, row in bad_p.iterrows():
    name = row['OBJECTID']    
    for inF in all_files:
        if str(name) == inF.split("_")[-1].split(".")[0]:
            os.remove(os.path.join(out_folder, inF))

In [ ]:
bad_p

# Run Market Access

In [ ]:
#selP = inP.loc[inP['REGION'] == 'South Asia']
inP = gpd.read_file(ports_file)

In [ ]:
travel_time_thresholds = [((24*60) * x) for x in [0.25,0.5,1,2]]
out_folder = os.path.join(os.path.dirname(ports_file), "TRAVEL_TIMES")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
port_files = []
for idx, row in inP.iterrows():
    name = row['OBJECTID']    
    out_file = os.path.join(out_folder, "PORT_ISOCHRONES_NO_OCEAN_%s.shp" % name)
    port_files.append(out_file)
    if not os.path.exists(out_file):
        print("Processing %s" % name)
        current_p = pd.DataFrame(inP.loc[idx,]).transpose()
        travel_times = ma.generate_feature_vectors(inG, mcp, current_p, travel_time_thresholds)
        travel_times.to_file(out_file)
    else:
        print("%s already processed" % name)
        
    

In [ ]:
port_files

In [ ]:
try:
    del final
except:
    pass
for p in port_files:
    current_p = gpd.read_file(p)
    current_p['PORT'] = p.split("_")[-1].replace(".shp","")
    try:
        final = final.append(current_p)
    except:
        final = current_p

In [ ]:
final.to_file(os.path.join(out_folder, "COMBO_all_ports.shp"))

In [ ]:
out_folder

# Zonal Stats

In [ ]:
inShape = os.path.join(out_folder, "COMBO_all_ports.shp")